In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import nibabel as nib

from scipy.stats import zscore, pearsonr, kendalltau,  entropy, spearmanr, linregress, rankdata, ttest_rel, ttest_1samp, ttest_ind
from statsmodels.stats import multitest
import scipy.linalg as la
import itertools
import random
import math

In [3]:
DATADIR = '/home/jiunchoi/OFD/OFD_BHV_clean'
group = 2
sub = 1
groupsub = '0'+str(group)+'0'+str(sub)

In [4]:
def extract_recalledidx(group):
    keys = [f'sub-0{group}01',f'sub-0{group}02',f'sub-0{group}03']

    data_transcript = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{str(group)}/group-0{str(group)} sharedeb.xlsx')
    sharedeb = np.array(data_transcript['sharedeb'])
    data_recs = np.array(data_transcript[keys])

    recalled_evidx=[]
    recalled_evs = []
    for i in range(len(sharedeb)):
        count=0
        for j in range(3):
            if (type(data_recs[i,j])==str or np.isnan(data_recs[i,j])==False) and (data_recs[i,j]!=0):
                count+=1
        if count==3:
            if sharedeb[i] not in recalled_evidx:
                recalled_evidx.append(sharedeb[i])
                recalled_evs.append(data_recs[i,:])
    return recalled_evs

def get_boundary(group,sub,run):
    groupsub = '0'+str(group)+'0'+str(sub)
    word_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_AUDIO/derivatives/group-0{group}/sub-{groupsub}_run-{run}_day2_words.xlsx')
    posthoc_data = pd.read_excel(f'/home/jiunchoi/OFD/OFD_BHV_clean/group-0{group}/sub-{groupsub}_day2_posthoc.xlsx')
    # word_data.keys() = start, end, word
    # posthoc_data.keys() = sentence, segmentation, description, tag
    words = word_data['word']
    start = word_data['start']
    end = word_data['end']
    sentstartend = []
    eventstartend = []
    endidx = [0]
    for t in range (len(posthoc_data['Unnamed: 0'])-1):
        if posthoc_data['Unnamed: 0'][t]+1 != posthoc_data['Unnamed: 0'][t+1]:
            endidx.append(t)
    endidx.append(len(posthoc_data['Unnamed: 0'])-1)
    if len(endidx)>2:
        segcol = np.array(posthoc_data['segmentation'][endidx[run-1]+1:endidx[run]+1])
    else:
        segcol = np.array(posthoc_data['segmentation'])

    tmpstart = start[0]
    for i in range (len(words)-1):
        if '.' in str(words[i]) or ',' in str(words[i]) or '었고' in str(words[i]) or '것 같고' in str(words[i]) or '지고' in str(words[i]) or '는데' in str(words[i]) or '근데' in str(words[i]) or '그리고' in str(words[i]) or '그러고' in str(words[i]) or '됐고' in str(words[i]) or '했고' in str(words[i]): 
            tmpend = end[i]
            sentstartend.append([tmpstart,tmpend])
            tmpstart = start[i+1]
    tmpend = end[len(words)-1]
    sentstartend.append([tmpstart,tmpend])

    # working for subs with only 1 run
    tmpstart = sentstartend[0][0]
    for i in range (len(segcol)-1):
        if segcol[i] != segcol[i+1]:     
            tmpend = sentstartend[i][1]     
            eventstartend.append([tmpstart,tmpend])
            tmpstart = sentstartend[i+1][0]
    tmpend = sentstartend[len(segcol)-1][1]
    eventstartend.append([tmpstart,tmpend]) 
    
    return eventstartend

def get_motion_params(subn,task,run):
    fname = f'/home/jiunchoi/OFD/OFD_DATA/derivatives/sub-{subn}/func/sub-{subn}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'
    compounds = pd.read_csv(fname, delimiter='\t')

    confound_labels =  ['trans_x', 
                    'trans_y', 
                    'trans_z',
                    'rot_x',
                    'rot_y',
                    'rot_z',
                    'trans_x_derivative1',
                    'trans_y_derivative1',
                    'trans_z_derivative1',
                    'rot_x_derivative1',
                    'rot_y_derivative1',
                    'rot_z_derivative1',
                    'trans_x_power2',
                    'trans_y_power2',
                    'trans_z_power2',
                    'rot_x_power2',
                    'rot_y_power2',
                    'rot_z_power2',
                    'trans_x_derivative1_power2',
                    'trans_y_derivative1_power2',
                    'trans_z_derivative1_power2',
                    'rot_x_derivative1_power2',
                    'rot_y_derivative1_power2',
                    'rot_z_derivative1_power2']
    X_motion = compounds[confound_labels]
    X_motion = np.nan_to_num(X_motion)
    return X_motion

def mot_regout(groupsub,task,run,data):
    motparam = get_motion_params(groupsub, task, run)

    data_mean = data.mean(0)
    data = data - data_mean
    coef, _, _, _ = la.lstsq(motparam, data)
    # remove trends and add back mean of the data
    data_clean = data - motparam.dot(coef) + data_mean

    return data_clean


MASKDIR = '/home/jiunchoi/OFD/source/'
DATADIR = '/home/jiunchoi/OFD/OFD_DATA/derivatives/'
BN_atlas = nib.load(f'{MASKDIR}BNA_3mm_atlas.nii').get_fdata()

def load_brain(group,sub,run):
    task = 'RECALL'
    BN_parcels = np.zeros((246), dtype='object')
    groupsub = '0'+str(group)+'0'+str(sub)
    fmri_data = nib.load(f'{DATADIR}sub-{groupsub}/preprocessed/sub-{groupsub}_{task}_{run}_sc_dt_sm.nii.gz').get_fdata()
    for roi in range (1,246+1):
        roi_data = fmri_data[BN_atlas==roi, :].T #(Time, Voxels)
        roi_data = zscore(roi_data, axis=0)
        roi_data = np.nan_to_num(roi_data)
        BN_parcels[roi-1] = roi_data
    return BN_parcels

In [5]:
groups=[2,3,4,5]
group_brain = np.zeros((len(groups),3), dtype='object')

for group in groups:
    for sub in [1,2,3]:
        sub_brain = []
        if (group==2 and sub==1):
            runs = [1,2,3]
        else:
            runs = [1]
        for run in runs:
            sub_brain.append(load_brain(group,sub,run))
        group_brain[group-2,sub-1] = sub_brain

In [6]:
groups=[2,3,4,5]; roi=245

group_evpatterns = []
for group in groups:
    recalled_evs = extract_recalledidx(group)
    event_patterns = np.zeros_like(recalled_evs,dtype='object')
    for sub in [1,2,3]:
        if (group==2 and sub==1):
            runs = [1,2,3]
        else:
            runs = [1]
        eventstartend = []
        for run in runs:
            eventstartend.extend(get_boundary(group,sub,run))

        for e in range(len(recalled_evs)):
            ev_from_recall = str(recalled_evs[e][sub-1]).split(',')
            if len(ev_from_recall) > 1:
                single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                for efr in ev_from_recall[1:]:
                    single_pattern_timepoint = np.vstack((single_pattern_timepoint,eventstartend[int(efr)-1]))
                single_pattern = group_brain[group-2,sub-1][run-1][roi][int(single_pattern_timepoint[0,0]/1000):int(single_pattern_timepoint[0,1]/1000),:]
                for s in range(1, single_pattern_timepoint.shape[-1]):
                    single_pattern = np.vstack((single_pattern,group_brain[group-2,sub-1][run-1][roi][int(single_pattern_timepoint[s,0]/1000):int(single_pattern_timepoint[s,1]/1000),:]))
            else:
                single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                single_pattern = group_brain[group-2,sub-1][run-1][roi][int(single_pattern_timepoint[0]/1000):int(single_pattern_timepoint[1]/1000),:]
            event_pattern = np.mean(single_pattern,axis=0)
            event_patterns[e,sub-1] = event_pattern
    group_evpatterns.append(event_patterns)

/home/jiunchoi/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jiunchoi/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/jiunchoi/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jiunchoi/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [150]:
idx = [0,1,2]
eventwise_rs = np.zeros((246,len(groups)),dtype='object')

menu_evs = [[8],[10,11,12,14,15,16,17],[3,4],[5,6,7,8,9,10,11,12]] #순서대로 group
task2_evs = [[9,10,11,12,13,14,15,16],[8,9,18,19],['nan'],[4,14,15,16]]
task13_evs = []

for g,group in enumerate(groups):
    recalled_evs = extract_recalledidx(group)
    task13_ev = [i for i in range(len(recalled_evs))]
    subtracted = [x for x in task13_ev if x not in menu_evs[g] and task2_evs[g]]
    task13_evs.append(subtracted)

entire_roigroups_evpatterns = np.zeros((246,len(groups)),dtype='object') #roi, group, sub, event

for roi in range(246):
    for group in groups:
        recalled_evs = extract_recalledidx(group)
        event_patterns = np.zeros_like(recalled_evs,dtype='object')
        for sub in [1,2,3]:
            if (group==2 and sub==1):
                runs = [1,2,3]
            else:
                runs = [1]
            eventstartend = []
            runidx = []
            for run in runs:
                evse = get_boundary(group,sub,run)
                eventstartend.extend(evse)
                runidx.extend([run]*len(evse))
                
            for e in range(len(recalled_evs)):
                ev_from_recall = str(recalled_evs[e][sub-1]).split(',')

                if len(ev_from_recall) > 1:
                    single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                    for efr in ev_from_recall[1:]:
                        single_pattern_timepoint = np.vstack((single_pattern_timepoint,eventstartend[int(float(efr))-1]))
                    single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0,0]/1000):math.ceil(single_pattern_timepoint[0,1]/1000),:]
                    for s in range(1, single_pattern_timepoint.shape[-1]):
                        single_pattern = np.vstack((single_pattern,group_brain[group-2,sub-1][runidx[math.trunc(float(ev_from_recall[0]))-1]-1][roi][math.ceil(single_pattern_timepoint[s,0]/1000):int(single_pattern_timepoint[s,1]/1000),:]))
                else:
                    single_pattern_timepoint = eventstartend[int(float(ev_from_recall[0]))-1]
                    single_pattern = group_brain[group-2,sub-1][runidx[int(float(ev_from_recall[0]))-1]-1][roi][math.trunc(single_pattern_timepoint[0]/1000):math.ceil(single_pattern_timepoint[1]/1000),:]
                event_pattern = np.mean(single_pattern,axis=0)
                event_patterns[e,sub-1] = event_pattern
            entire_roigroups_evpatterns[roi,group-2] = event_patterns

In [156]:
# calculate ISPC
eventwise_rs = np.zeros((246,len(groups)),dtype='object')
for roi in range(246):
    for group in groups:
        eventwise_r = np.zeros((len(entire_roigroups_evpatterns[roi][group-2]),3))
        for e in range(len(eventwise_r)):
            for j in range (3):
                a = idx[j-2]
                b = idx[j-1]
                c = idx[j]
            
                pattern_1 = entire_roigroups_evpatterns[roi][group-2][e,a]
                pattern_2 = np.mean((entire_roigroups_evpatterns[roi][group-2][e,b],entire_roigroups_evpatterns[roi][group-2][e,c]),axis=0)
                eventwise_r[e,a] = pearsonr(pattern_1, pattern_2)[0]
        eventwise_rs[roi,group-2] = eventwise_r

In [195]:
n_permutation = 1000
permuted_stats = np.zeros((n_permutation,2))

for i in range(n_permutation):
    chosen_evs = np.zeros((4,3)) #group, conds
    chosen_evpatterns = np.zeros((4,3,2)) #group, rs, conditions
    for group in groups:
        chosen_evs[group-2,0] = random.choice(menu_evs[group-2]) #menu
        # chosen_evs[group-2,1] = random.choice(task2_evs[group-2]) #task2
        chosen_evs[group-2,2] = random.choice(task13_evs[group-2]) #task13
        chosen_evpatterns[group-2,:,0] = eventwise_rs[roi,group-2][int(chosen_evs[group-2,0])]
        chosen_evpatterns[group-2,:,1] = eventwise_rs[roi,group-2][int(chosen_evs[group-2,2])]

    group1 = chosen_evpatterns[:,:,0].flatten()
    group2 = chosen_evpatterns[:,:,1].flatten()

    t_stat, p_values = stats.ttest_ind(group1, group2)
    corrected_p_values = multitest.multipletests(p_values, method='fdr_bh')[1]
    permuted_stats[i,:] = [tstat, corrected_p_values]